In [1]:
from priomptipy import SystemMessage, UserMessage, AssistantMessage, Scope, render
from pprint import pprint
import pandas as pd

In [2]:
def add_few_shot(examples: list[dict]) -> list[Scope]:
    """Converting examples to the required structure.

    Args:
        examples (list[dict]): A list of dictionaries
        containing input and output examples.

    Returns:
        list[Scope]: A list of Scope objects containing
        the few shot examples.
    """
    few_shot_scope = []
    for ex in examples:
        # Create a Scope object for each example with highest priority
        few_shot_scope.append(Scope([
            UserMessage(ex["input"]),
            AssistantMessage(ex["output"])
        ], absolute_priority=10))
    return few_shot_scope

In [3]:
sales_data = {
    'Product': ['A', 'B', 'C', 'D', 'E'],
    'Sales (Jan)': [100, 80, 50, 90, 200],
    'Sales (Feb)': [150, 90, 60, 100, 210],
    'Sales (Mar)': [200, 120, 70, 110, 220],
    'Sales (Apr)': [250, 130, 80, 120, 230],
    'Sales (May)': [300, 160, 100, 130, 240],
    'Sales (Jun)': [350, 200, 110, 140, 250]
}
sales_df = pd.DataFrame(sales_data)
df_as_text = sales_df.to_string(index=False)


In [4]:
few_shot_data = [
    {"input": "Analyze the sales trend for Product B over six months.",
     "output": "Product B shows a steady growth over the six-month period.\
        Sales grew from 80 in January to 200 in June, indicating a total increase of 150%."},
    
    {"input": "Which product had the highest total sales over the last six months?",
     "output": "Product E had the highest total sales over six months, with a combined total of 1,350 units."},
    
    {"input": "Compare the growth rates of Products A and C over the six-month period.",
     "output": "Product A grew by 250%, while Product C grew by 120%. Product A exhibited a stronger growth rate."},
    
    {"input": "What is the average monthly sales for Product D?",
     "output": "The average monthly sales for Product D is 115 units."}
]
few_shot_examples = add_few_shot(few_shot_data)

In [5]:
system_message = [SystemMessage("You are Quarkle, an AI Developmental Editor")]

actual_conversation = [UserMessage(f"Here is the sales data:\n{df_as_text}\n\
                                   Calculate the percentage increase in sales for Product E from January to June.")]

In [6]:
# Combine all message components
messages = system_message + few_shot_examples + actual_conversation
# Set rendering options including token limit and tokenizer
render_options = {"token_limit": 1000, "tokenizer": "cl100k_base"}
# Render the messages
result = await render(messages, render_options)
pprint(result['prompt'])

{'messages': [{'content': 'You are Quarkle, an AI Developmental Editor',
               'role': 'system'},
              {'content': 'Analyze the sales trend for Product B over six '
                          'months.',
               'role': 'user'},
              {'content': 'Product B shows a steady growth over the six-month '
                          'period.        Sales grew from 80 in January to 200 '
                          'in June, indicating a total increase of 150%.',
               'role': 'assistant'},
              {'content': 'Which product had the highest total sales over the '
                          'last six months?',
               'role': 'user'},
              {'content': 'Product E had the highest total sales over six '
                          'months, with a combined total of 1,350 units.',
               'role': 'assistant'},
              {'content': 'Compare the growth rates of Products A and C over '
                          'the six-month period.',